In [49]:
import json
import sqlalchemy
import pandas as pd

The next few lines establish a connection to the database. The json credential file uses the following format:
{
    "database": "*****",
     "host": "*****",
     "port": 5432,
     "user": "*****",
     "password": "*****"
}

In [50]:
with open("/home/btphilip/notebooks/SQL_access.json") as f:
    db_conn_dict = json.load(f)

In [51]:
cred_string = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**db_conn_dict)

In [52]:
dbeng = sqlalchemy.create_engine(cred_string)

The next few lines read in a csv, selects a subset of the data, stores it as a new variable and prints the variable

In [53]:
df = pd.read_csv('/home/btphilip/snowex/SBBgps20170222A.csv')
temp_df = df[0:99]
temp_df.head(5)

,day.dec,latitude,N/S,longitude,E/W,GPSfix,NSats,HDOP,elevation
0,736382.673322,37.905610,N,107.707907,W,1,7,3.33,3344.7
1,736382.673333,37.905610,N,107.707903,W,1,7,3.33,3344.1
2,736382.673345,37.905609,N,107.707899,W,1,7,3.33,3343.6
3,736382.673356,37.905608,N,107.707896,W,1,7,3.33,3343.3
4,736382.673368,37.905607,N,107.707892,W,1,7,3.33,3342.7


The following line writes the temp_df variable to a database table specified by the db_table_name on the geohack server (stored here as dbeng). This server is hosted on AWS for the duration of Geohackweek 2017 and was created using the AWS RDS service. I've been using the dBeaver GUI to query and edit the database.  

In [54]:
db_table_name = 'philip_ice_2'
temp_df.longitude= temp_df['longitude'].copy()*-1
temp_df.head(3)
temp_df.to_sql(db_table_name, dbeng)

/opt/anaconda/envs/vectorenv/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Now create a new geometry column within the database table to fill with GeoDataFrame values. Converting to geodataframe will allow for additional SQL queries (e.g. nearest values)

In [55]:
dbeng.execute("""ALTER TABLE %s ADD COLUMN geom geometry(Point, 32612);""" %(db_table_name)) 

Now populate the geometry field using ST_transform

In [56]:
dbeng.execute("""UPDATE %s SET geom = ST_Transform(ST_setSRID(ST_MakePoint(longitude,latitude),4326),32612);""" %(db_table_name))